In [51]:
# TODO account for main stat of 5-star
# nbd cuz false positives (of good runes) are unlikely:
# they're dragged down by low subs anyways
# TODO should have equal 1/3/5 and questionably equal 4/6?

In [52]:
# Config

IS_FRR = True

TIERS_TO_SETS = {'S': ['Violent'], 'A': ['Swift', 'Will'], 
'B+': ['Fatal', 'Rage', 'Vampire', 'Despair'],
'B-': ['Blade',  'Focus', 'Shield', 'Nemesis', 'Revenge', 'Destroy'],
'C': ['Energy', 'Guard', 'Endure', 'Fight', 'Determination', 'Enhance', 'Accuracy', 'Tolerance']}

GOAL_TOTAL = 3000 # start lower, but steadily raise to 3000.
TIER_RATIOS = {'S': 0.26, 'A': 0.26, 'B+': 0.21, 'B-': 0.17, 'C': 0.10}
IS_ODD_RATIO = 1/3
CUTOFF_FACTOR = 0.72 # all held runes can _potentially_ be better than the n-th best rune. 
# where is this cutoff? where do you draw the line? by some scale:
# what is n divided by the number of total runes of class?
# Higher factor means that you hold more runes, 
# Lower factor means holding less runes, thus less flexibility but saves mana (more selective and so less to powerup)
# (Note that if scaling_factor=1, then you will hold more runes than the goal_total:
# you'll hold everything already better, and potentially better)
# Fine-tune as needed
# A big guiding factor:
# ADJUST SUCH THAT MANA STONES ARE MANAGEABLE 
# ADJUST SUCH THAT ACTUAL HELD RUNE COUNTS / FREQS MATCH GOAL COUNTS / FREQS

GLOBAL_EFF_CUTOFF = 7 # any non-spd rune, less than this max eff are sold, regardless of the goal totals etc
# To prevent flooding inventory with runes that, even if they're _relatively_ good enough,
# would still be easily, quickly sold by obtaining better runes

# Compute proportion of quantity that slots should take.
# !! DOES NOT INCLUDE SPD RUNES
# Note: balance btwn 2 (non-spd) vs 4 and 6 is based on empirical data
# adjust as needed
SLOT_RATIOS = {1: 1/9, 3: 1/9, 5: 1/9, 2: 0.22, 4: 0.39, 6: 0.39}

In [53]:
import numpy as np
import pandas as pd

In [54]:
weights = {'sub_acc': 1, 'sub_res': 1, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 1, 'sub_deff': 0.5, 'sub_hpp': 1, 'sub_hpf': 0.5, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}
weights_off = {'sub_acc': 1, 'sub_res': 0.5, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 0.5, 'sub_deff': 0.25, 'sub_hpp': 0.5, 'sub_hpf': 0.25, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}
weights_def = {'sub_acc': 1, 'sub_res': 1, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 1, 'sub_deff': 0.5, 'sub_hpp': 1, 'sub_hpf': 0.5, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}

In [55]:
df = pd.read_csv('runes-data.csv', sep=';')

In [56]:
# Clean entries
df = df.replace('-', np.nan)

In [57]:
# For rune counting, drop Tricaru Icaru runes
df = df[~(df['monster_n'].str.contains('Icaru'))]

In [58]:
# Count rune sets at a glance
# Good for adjusting cutoffs
df_not_inventory = df[~(df['monster_n'] == 'Inventory')]
print(len(df_not_inventory))
df_not_inventory.value_counts('set')

599


set
Violent          148
Swift            100
Will              55
Fatal             40
Vampire           36
Despair           31
Rage              28
Blade             24
Revenge           24
Energy            20
Focus             18
Destroy           16
Guard             15
Nemesis           15
Shield            11
Endure             6
Fight              5
Tolerance          3
Enhance            2
Determination      1
Accuracy           1
dtype: int64

In [59]:
# Clean columns
df = df.drop(columns=['s1_t',	's1_v',	's1_data',
                      's2_t', 's2_v', 's2_data',
                      's3_t',	's3_v',	's3_data',
                      's4_t',	's4_v',	's4_data',
                      'DT_RowId',	'id',	'unique_id',	'monster',	'originID', 'originName', 'efficiency', 'max_efficiency', 'locked'])

cols_original_clean = df.columns

In [60]:
# convert inherent stats to eff values, 0-1.
# keep separate because these can't be increased like normal stats

inherent_label_to_sub_label = {'ACC': 'sub_acc', "RES": "sub_res", "ATK%": "sub_atkp", "ATK flat": "sub_atkf",  "DEF%": "sub_defp", "DEF flat": "sub_deff",  "HP%": "sub_hpp", "HP flat": "sub_hpf", "SPD": "sub_spd", "CRate": "sub_crate", "CDmg": "sub_cdmg"}

df['i_t_clean'] = df['i_t'].replace(inherent_label_to_sub_label)

substats_max = {'sub_acc':8, 'sub_res': 8, 'sub_atkp': 8, 'sub_atkf': 20, 'sub_defp': 8, 'sub_deff': 20, 'sub_hpp': 8, 'sub_hpf': 375, 'sub_spd': 6, 'sub_crate': 6, 'sub_cdmg': 7}

df['inh_norm'] = df['i_v'] / (df['i_t_clean'].replace(substats_max)) * (df['i_t_clean'].replace(weights)) 

In [61]:
# convert substats to eff values, 0-5
for label in substats_max:
    df[label] = pd.to_numeric(df[label])

for label in substats_max:
    df[label+'_norm'] = df[label]/substats_max[label]*weights[label]


In [62]:
#specify the columns to sum
cols = [str(label+'_norm') for label in substats_max]
cols.append('inh_norm')
#find sum of columns specified 
df['tot_sum_norm'] = df[cols].sum(axis=1)

In [63]:
df['num_powerup_left'] = np.maximum(0, np.subtract(4, np.floor_divide(df['level'], 3)))
df['num_powerup_used'] = np.minimum(4, np.floor_divide(df['level'], 3))


df['num_powerup_incsub'] = df['quality'].map({'Unknown': 0, 'Rare': 2, 'Hero': 3, 'Legend': 4})

df['num_powerup_incsub_left'] = np.maximum(np.subtract(df['num_powerup_incsub'], df['num_powerup_used']) , 0)
df['num_powerup_newsub_left'] = np.subtract(df['num_powerup_left'], df['num_powerup_incsub_left'] )

In [64]:
# From increasing current substats:
# if spd is an increasable substat, then assume all rolls go there
# otherwise assume all rolls go to not-good stats
# TODO this roll could possibly only go to a bad stat (as per norm), not a good one. account for this

df['sub_inc_max_norm'] = (df['num_powerup_incsub_left']*2).where(~df['sub_spd'].isna(), df['num_powerup_incsub_left'])

# assume these are all going to bad stats.
# (even if speed is rollable, probability of going to spd is...low and not worth)
# TODO consider like max vs expected. this straddles the line somewhere
df['sub_new_max_norm'] = df['num_powerup_newsub_left']

In [65]:
# and now sum for the "max" roll eff
cols = ['tot_sum_norm', 'sub_inc_max_norm', 'sub_new_max_norm']

df['tot_max_norm'] = df[cols].sum(axis=1)

In [66]:
# worst cases:
# TODO due to the chance of flat rolls, this is lower.
# also like account for inc rolls can only be present stats; new rolls can only be one of each (not double-up into bad roll)
df['sub_inc_min_norm'] = df['num_powerup_incsub_left'] * 0.5 # (0.5 is worstcase eff from a roll)
df['sub_new_min_norm'] = df['num_powerup_newsub_left'] * 0.5
cols = ['tot_sum_norm', 'sub_inc_min_norm', 'sub_new_min_norm']
df['tot_min_norm'] = df[cols].sum(axis=1)

In [67]:
SETS_TO_TIERS = dict()
for tier in TIERS_TO_SETS:
    for set in TIERS_TO_SETS[tier]:
        SETS_TO_TIERS[set] = tier
print(SETS_TO_TIERS)

df['tier'] = df['set'].apply(lambda x: SETS_TO_TIERS[x])

# To scale to a normalized 0-to-1 scale
# df['tot_max_norm'] = (df['tot_max_norm']/9.round(3)

{'Violent': 'S', 'Swift': 'A', 'Will': 'A', 'Fatal': 'B+', 'Rage': 'B+', 'Vampire': 'B+', 'Despair': 'B+', 'Blade': 'B-', 'Focus': 'B-', 'Shield': 'B-', 'Nemesis': 'B-', 'Revenge': 'B-', 'Destroy': 'B-', 'Energy': 'C', 'Guard': 'C', 'Endure': 'C', 'Fight': 'C', 'Determination': 'C', 'Enhance': 'C', 'Accuracy': 'C', 'Tolerance': 'C'}


In [68]:
df['flag'] = False


for tier in TIER_RATIOS.keys():
    for slot in (1, 2, 3, 4, 5, 6):
        print(f'tier {tier} slot {slot}')

        goal_freq = int(GOAL_TOTAL * TIER_RATIOS[tier] * SLOT_RATIOS[slot])
        cutoff_freq = int(goal_freq * CUTOFF_FACTOR)

        mask_subset = (df['slot'] == slot) & (df['tier']==tier)

        # Figure out how many runes to sell: 
        # this is tricky, as we want to keep _all_ speed runes 
        # (due to quirks w efficiency, importance of speed, etc)
        # if not is_odd:
        #     count_spd = sum(mask_subset & (df['m_t'] == 'SPD'))
        #     cutoff_freq -= count_spd
        #     goal_freq -= count_spd
        #     mask_subset = mask_subset & (df['m_t'] != 'SPD')

        # TODO: this keeps all speed runes and 
        # does NOT include them in the 3000 count, which is odd
        mask_subset = mask_subset & (df['m_t'] != 'SPD')

        df_subset = df[mask_subset]
        df_subset = df_subset.sort_values(by=['tot_min_norm'], ascending=False)

        if len(df_subset) < cutoff_freq:
            # then want to keep everything. can't sell runes below the n'th-best if there aren't n total 
            # (effectively mark this by setting the "sell if below" threshold to 0)
            eff_cutoff = 0
        else:
            # Calc cutoff, based on rune counts, and "n-th" best rune
            rune_cutoff = df_subset.iloc[cutoff_freq-1]
            eff_cutoff = rune_cutoff.tot_min_norm
        # if eff cutoff less than global eff cutoff,
        # then enforce global instead
        print(f'Sell runes below global cut or {eff_cutoff} eff')
        if eff_cutoff < GLOBAL_EFF_CUTOFF:
            eff_cutoff = GLOBAL_EFF_CUTOFF

        mask_subset_to_sell = (mask_subset & (df['tot_max_norm'] < eff_cutoff))
        print(f'Have {sum(mask_subset)-sum(mask_subset_to_sell)} runes, goal {goal_freq}; diff {goal_freq-(sum(mask_subset)-sum(mask_subset_to_sell))}')

        # flag runes in the subset, below the cutoff
        # keep previous runes too as flagged ofc: flag builds
        df['flag'] = (mask_subset_to_sell) | df['flag']

        print()


tier S slot 1
Sell runes below global cut or 0 eff
Have 25 runes, goal 86; diff 61

tier S slot 2
Sell runes below global cut or 0 eff
Have 9 runes, goal 171; diff 162

tier S slot 3
Sell runes below global cut or 0 eff
Have 36 runes, goal 86; diff 50

tier S slot 4
Sell runes below global cut or 0 eff
Have 15 runes, goal 304; diff 289

tier S slot 5
Sell runes below global cut or 0 eff
Have 18 runes, goal 86; diff 68

tier S slot 6
Sell runes below global cut or 0 eff
Have 16 runes, goal 304; diff 288

tier A slot 1
Sell runes below global cut or 0 eff
Have 51 runes, goal 86; diff 35

tier A slot 2
Sell runes below global cut or 0 eff
Have 13 runes, goal 171; diff 158

tier A slot 3
Sell runes below global cut or 6.19047619047619 eff
Have 67 runes, goal 86; diff 19

tier A slot 4
Sell runes below global cut or 0 eff
Have 31 runes, goal 304; diff 273

tier A slot 5
Sell runes below global cut or 0 eff
Have 46 runes, goal 86; diff 40

tier A slot 6
Sell runes below global cut or 0 eff
H

## Mark some runes for reapp

 [S-or-A tier and Original Legend and 246]

In [69]:
mask_reapp_candidates = (df['tier'].isin(['S', 'A'])) & (df['quality']=='Legend') & (df['slot'].isin([2, 4, 6]))
df['sell'] = df['flag'] & ~mask_reapp_candidates
df['reapp'] = df['flag'] & mask_reapp_candidates

# Export: format data

In [70]:
cols_export = cols_original_clean
cols_export = cols_export.append(pd.Index(['tot_sum_norm', 'tot_min_norm', 'tot_max_norm', 'tier', 'num_powerup_incsub_left', 'flag', 'sell', 'reapp']))
df_export = df[cols_export]

In [71]:
df_export = df_export.sort_values(['set', 'grade', 'slot'], ascending=[True, False, True])

In [72]:
import datetime
  
# ct stores current time
ct = datetime.datetime.now()
ct = int(ct.timestamp())
df_export.to_csv(f'runes-data-out-{str(ct)}.csv', sep='\t')

In [73]:
df_export_to_sell = df_export[df_export['sell']]
df_export_to_sell.to_csv(f'runes-data-sell-{str(ct)}.csv', sep='\t')

df_export_to_reapp = df_export[df_export['reapp']]
df_export_to_reapp.to_csv(f'runes-data-reapp-{str(ct)}.csv', sep='\t')